In [1]:
import os
from pathlib import Path

print("🧹 1. เคลียร์พื้นที่และโหลดโค้ดใหม่...")
if os.path.exists("3DSCAN"):
    !rm -rf 3DSCAN
!git clone https://github.com/PRIDA-TAKON/3DSCAN.git
os.chdir("3DSCAN")

# ---------------------------------------------------------
# ✅ NOTE: Patches are no longer needed.
# The 3d-scan.py script has been updated to handle dependencies correctly.
# ---------------------------------------------------------

print(f"\n🚀 5. เริ่มรัน 3d-scan.py ...")
print("-" * 50)
!python 3d-scan.py 2>&1 | tee 3d-scan.log.txt

🧹 1. เคลียร์พื้นที่และโหลดโค้ดใหม่...
Cloning into '3DSCAN'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 104 (delta 37), reused 17 (delta 7), pack-reused 30 (from 1)
Receiving objects: 100% (104/104), 962.86 KiB | 2.55 MiB/s, done.
Resolving deltas: 100% (47/47), done.

🚀 5. เริ่มรัน 3d-scan.py ...
--------------------------------------------------
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.8.4 requires fastcore<1.9,>=1.8.0, but you have fastcore 1.11.3 which is incompatible.
  Installing build dependencies: started
  Installi